In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import timedelta
import datetime
import pytz

In [4]:
spx = pd.read_parquet("SPX.parquet")
vix = pd.read_parquet("VIX4y.parquet")

In [5]:
spx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754 entries, 0 to 8753
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype                        
---  ------    --------------  -----                        
 0   date      8754 non-null   datetime64[ns, Europe/Berlin]
 1   open      8754 non-null   float64                      
 2   high      8754 non-null   float64                      
 3   low       8754 non-null   float64                      
 4   close     8754 non-null   float64                      
 5   volume    8754 non-null   float64                      
 6   average   8754 non-null   float64                      
 7   barCount  8754 non-null   int64                        
dtypes: datetime64[ns, Europe/Berlin](1), float64(6), int64(1)
memory usage: 547.3 KB


In [6]:
spx["ma5"] = spx["close"].rolling(5).mean().round(2)
spx["ma20"] = spx["close"].rolling(20).mean().round(2)
spx_hourly = spx.iloc[19:]

In [13]:
test = spx_hourly.loc[spx_hourly["date"] > "2025-09-01"]

In [ ]:
def create_graph(df):
    layout = go.Layout(
    autosize=True
    )
    fig = go.Figure(data=[go.Candlestick(x=df["date"],
                                        open=df["open"],
                                        high=df["high"],
                                        low=df["low"],
                                        close=df["close"],
                                        name="SPX"),
                                        #add moving average
                                        go.Scatter(x=df["date"],
                                                y=df["ma5"],
                                                name="MA 5 hours")
                                                ])

    fig.update_xaxes(
        rangebreaks=[
        {"pattern" : "day of week", "bounds" : [6, 1]} #hides weekends
        {"pattern" : "hour", "bounds": [17,9]}
        ]
    )
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig

In [17]:
fig = create_graph(test)

In [18]:
fig.show()

In [10]:
layout = go.Layout(
    autosize=True
)
fig = go.Figure(data=[go.Candlestick(x=spx_hourly["date"],
                                    open=spx_hourly["open"],
                                    high=spx_hourly["high"],
                                    low=spx_hourly["low"],
                                    close=spx_hourly["close"],
                                    name="SPX"),
                                    #add moving average
                                    go.Scatter(x=spx_hourly["date"],
                                               y=spx_hourly["ma5"],
                                               name="MA 5 hours")
                                               ])

fig.update_xaxes(
    rangebreaks=[
    {"pattern" : "day of week", "bounds" : [6, 1]}
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)


In [22]:
fig.show()

In [3]:
spx = spx.set_index("date")
vix = vix.set_index("date")

In [4]:

spx_1h = pd.DataFrame(spx["close"].resample("1h").mean().dropna().astype("float32"))
vix_1h = vix["close"].resample("1h").mean().dropna().astype("float32")
#spx_1d = spx["close"].resample("1d").mean().dropna().astype("float32")

In [5]:
spx_1h.info(), vix_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8754 entries, 2020-10-19 15:00:00+02:00 to 2025-10-16 20:00:00+02:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   8754 non-null   float32
dtypes: float32(1)
memory usage: 102.6 KB
<class 'pandas.core.series.Series'>
DatetimeIndex: 14029 entries, 2021-10-18 09:00:00+02:00 to 2025-10-16 20:00:00+02:00
Series name: close
Non-Null Count  Dtype  
--------------  -----  
14029 non-null  float32
dtypes: float32(1)
memory usage: 164.4 KB


(None, None)

In [6]:
spx_1h["ma5"] = spx_1h["close"].rolling(5).mean().round(2)
spx_1h["ma20"] = spx_1h["close"].rolling(20).mean().round(2)
#spx_1h["ma20"] = spx_1h.rolling(20).mean().round(2)

In [7]:
spx_1h[19:]

,close,ma5,ma20
date,,,
2020-10-21 20:00:00+02:00,3457.790039,3445.47,3455.28
2020-10-21 21:00:00+02:00,3435.129883,3442.71,3452.34
2020-10-22 15:00:00+02:00,3443.590088,3443.36,3450.52
2020-10-22 16:00:00+02:00,3430.939941,3442.19,3448.49
2020-10-22 17:00:00+02:00,3431.340088,3439.76,3446.52
...,...,...,...
2025-10-16 16:00:00+02:00,6693.450195,6680.12,6664.41
2025-10-16 17:00:00+02:00,6666.810059,6680.33,6664.48
2025-10-16 18:00:00+02:00,6656.060059,6675.59,6664.51


In [8]:
spx_1h = spx_1h.join(vix_1h, how="left", rsuffix="_vix").dropna()

In [9]:
start_dates = [d.date() for d in spx_1h.index if d.weekday() == 4]
start_dates = sorted(list(set(start_dates)))


In [12]:
start = start_dates[100]
stop = start + timedelta(weeks=14)
test = datetime.datetime(start.year, start.month, start.day)

In [23]:
test

datetime.datetime(2023, 10, 13, 0, 0)

In [39]:
fig = px.line(spx_1h.tz_localize(None).loc[test:test + timedelta(weeks=13)].reset_index(), y=["close", "ma5", "ma20"], title="SPX_1h_lines")
fig.show()